# Machine Learning for Seagrass Extent Prediction

In [9]:
from pystac.client import Client
from odc.stac import load
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import joblib
from matplotlib import colors


from utils import scale, apply_masks, do_prediction, calculate_band_indices

## Loading Sentinel-2 GeoMAD

Load data, then create band indices and mask out areas we don't want to include.

In [10]:
catalog = "https://stac.digitalearthpacific.org"
client = Client.open(catalog)

In [11]:
# Ba Estuary
# bbox = [177.51971, -17.49416, 177.68452, -17.34430]
# datetime="2024"

In [12]:
# Komave
bbox = [177.72578562645654, -18.240565454997043, 177.8126842310554, -18.217513820319642]
datetime="2024"

In [13]:
items = client.search(
    collections=["dep_s2_geomad"], datetime=datetime, bbox=bbox
).item_collection()

print(f"Found {len(items)} items")

Found 1 items


In [14]:
data = load(
    items,
    bbox=bbox,
    measurements=[
        "nir",
        "red",
        "blue",
        "green",
        "emad",
        "smad",
        "bcmad",
        "count",
        "green",
        "nir08",
        "nir09",
        "swir16",
        "swir22",
        "coastal",
        "rededge1",
        "rededge2",
        "rededge3",
    ],
    chunks={"x": 2048, "y": 2048},
)

data

<xarray.Dataset> Size: 10MB
Dimensions:      (y: 269, x: 968, time: 1)
Coordinates:
  * y            (y) float64 2kB -2.05e+06 -2.05e+06 ... -2.052e+06 -2.052e+06
  * x            (x) float64 8kB 3.086e+06 3.086e+06 ... 3.096e+06 3.096e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/16)
    nir          (time, y, x) uint16 521kB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    red          (time, y, x) uint16 521kB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    blue         (time, y, x) uint16 521kB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    green        (time, y, x) uint16 521kB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    emad         (time, y, x) float32 1MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    smad         (time, y, x) float32 1MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    ...           ...
    swir16       (time, y, x) uint16 521kB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    swir22       (time, y, x) uint16 521kB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    coastal      (time, y, x) uint16 521kB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    rededge1     (time, y, x) uint16 521kB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    rededge2     (time, y, x) uint16 521kB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    rededge3     (time, y, x) uint16 521kB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>

In [15]:
scaled_data = scale(data)
indices = calculate_band_indices(scaled_data)

# TODO: split masks into separate functions
masked_data = apply_masks(scaled_data)

masked_data

<xarray.Dataset> Size: 53MB
Dimensions:      (time: 1, y: 269, x: 968)
Coordinates:
  * y            (y) float64 2kB -2.05e+06 -2.05e+06 ... -2.052e+06 -2.052e+06
  * x            (x) float64 8kB 3.086e+06 3.086e+06 ... 3.096e+06 3.096e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/27)
    nir          (time, y, x) float64 2MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    red          (time, y, x) float64 2MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    blue         (time, y, x) float64 2MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    green        (time, y, x) float64 2MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    emad         (time, y, x) float32 1MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    smad         (time, y, x) float32 1MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    ...           ...
    b_g          (time, y, x) float64 2MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    b_r          (time, y, x) float64 2MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    mci          (time, y, x) float64 2MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    ndci         (time, y, x) float64 2MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    stumpf       (time, y, x) float64 2MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    ln_bg        (time, y, x) float64 2MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>

In [16]:
masked_data.odc.explore(bands=["red", "green", "blue"], vmin=0, vmax=0.3)

## Run predictions over our region

In [17]:
model = joblib.load("models/model-geomad-joined-data-rf-11042025.model")

In [18]:
loaded = masked_data.compute()

In [19]:
loaded

<xarray.Dataset> Size: 53MB
Dimensions:      (time: 1, y: 269, x: 968)
Coordinates:
  * y            (y) float64 2kB -2.05e+06 -2.05e+06 ... -2.052e+06 -2.052e+06
  * x            (x) float64 8kB 3.086e+06 3.086e+06 ... 3.096e+06 3.096e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/27)
    nir          (time, y, x) float64 2MB 0.0447 0.0466 0.0477 ... nan nan nan
    red          (time, y, x) float64 2MB 0.0449 0.049 0.0543 ... nan nan nan
    blue         (time, y, x) float64 2MB 0.065 0.0644 0.0685 ... nan nan nan
    green        (time, y, x) float64 2MB 0.0685 0.0683 0.074 ... nan nan nan
    emad         (time, y, x) float32 1MB 0.139 0.1388 0.1382 ... nan nan nan
    smad         (time, y, x) float32 1MB 6.146e-06 6.159e-06 ... nan nan
    ...           ...
    b_g          (time, y, x) float64 2MB 0.9489 0.9429 0.9257 ... nan nan nan
    b_r          (time, y, x) float64 2MB 1.448 1.314 1.262 ... nan nan nan
    mci          (time, y, x) float64 2MB 0.8309 0.8694 0.7109 ... nan nan nan
    ndci         (time, y, x) float64 2MB 0.09017 0.04483 0.1054 ... nan nan nan
    stumpf       (time, y, x) float64 2MB 2.808 2.746 2.67 2.503 ... nan nan nan
    ln_bg        (time, y, x) float64 2MB -0.05245 -0.0588 -0.07723 ... nan nan

In [20]:
predicted_da = do_prediction(loaded, model).astype(np.float32)
predicted_da

ValueError: X has 27 features, but RandomForestClassifier is expecting 25 features as input.

## Visualisation
Visualise the prediction with the 8 class colour map or the 10 class colour map below, respectively

In [ ]:
classes = [
    [1, "sediment", "#8c8c8c"],
    [2, "sand", "#fedd24"],
    [3, "rubble", "#f8ffb4"],
    [4, "seagrass", "#6df7dc"],
    [5, "seaweed", "#b9df6f"],
    [6, "coral", "#a011c3"],
    [7, "rock", "#804600"],
    [8, "deeps", "#011b61"],
]

values_list = [c[0] for c in classes]
color_list = [c[2] for c in classes]

# Build a listed colormap.
c_map = colors.ListedColormap(color_list)
bounds = values_list + [8]
norm = colors.BoundaryNorm(bounds, c_map.N)
predicted_da.odc.explore(cmap=c_map)

In [ ]:
classes = [
    [1, "sediment", "#8c8c8c"],
    [2, "sand", "#fedd24"],
    [3, "rubble", "#f8ffb4"],
    [4, "seagrass", "#6df7dc"],
    [5, "seaweed", "#b9df6f"],
    [6, "coral", "#a011c3"],
    [7, "rock", "#804600"],
    [8, "deeps", "#011b61"],
    [9, "mangrove", "#086a39"],
    [10, "land", "#00FFFFFF"],
]

values_list = [c[0] for c in classes]
color_list = [c[2] for c in classes]

# Build a listed colormap.
c_map = colors.ListedColormap(color_list)
bounds = values_list + [14]
norm = colors.BoundaryNorm(bounds, c_map.N)

In [ ]:
predicted_da.odc.explore(cmap=c_map)

In [28]:
# predicted_da.odc.write_cog("predictions/predicted_ba_estuary_joined_data_postcard_svm_22042025.tiff", overwrite=True)
predicted_da.odc.write_cog("predictions/predicted_komave_postcard_rf_24042025.tiff", overwrite=True)

PosixPath('predictions/predicted_komave_postcard_rf_24042025.tiff')

### Quantitative validation (to be completed)

In [29]:
test_data = gpd.read_file("testing-data/utanglang_postcard.geojson")
test_data.explore()

In [3]:
print("stop here")

stop here


In [4]:
stop_here


NameError: name 'stop_here' is not defined

In [30]:
# First transform the training points to the same CRS as the data
test = test_data.to_crs(postcard.odc.geobox.crs)
# Next get the X and Y values out of the point geometries
training_da = test.assign(x=test.geometry.x, y=test.geometry.y).to_xarray()
# Now we can use the x and y values (lon, lat) to extract values from the median composite
training_values = (
    postcard.sel(training_da[["x", "y"]], method="nearest").squeeze().compute().to_pandas()
)
len(training_values)

# Join the training data with the extracted values and remove unnecessary columns
training_array = pd.concat([test["observed_id"], training_values], axis=1)
training_array = training_array.drop(
    columns=[
        "y",
        "x",
        "spatial_ref",
    ]
)
# # Drop rows where there was no data available
# training_array = training_array.dropna()
# Preview our resulting training array
training_array.head()

NameError: name 'postcard' is not defined

In [ ]:
# Join the training data with the extracted values and remove unnecessary columns
training_array = pd.concat([test["observed_id"], training_values], axis=1)
training_array = training_array.drop(
    columns=[
        "y",
        "x",
        "spatial_ref",
    ]
)
# # Drop rows where there was no data available
# training_array = training_array.dropna()
# Preview our resulting training array
training_array.head()

In [ ]:
print(predicted_da.dtype)  # Check the dtype of your DataArray
predicted_da = predicted_da.astype('float32')  # Convert to float32

In [ ]:
print(len(training_array), len(test))  # Check the lengths of both arrays

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

# Sample data
np.random.seed(42)
training_array = np.random.rand(100, 5)
test = pd.DataFrame({"observed_id": np.random.choice([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 100)})

# Split the data
X_train, X_test, y_train, y_test = train_test_split(training_array, test.observed_id, test_size=0.9, random_state=42)

# Replace None values with a default value, e.g., 0 or the most frequent value
y_train = y_train.fillna(0)  # or y_train.fillna(y_train.mode()[0])
y_test = y_test.fillna(0)    # Ensure y_test also has no None values
y_train = y_train.astype(int)
y_test = y_test.astype(int)

# Train your model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Generate and display confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_display = ConfusionMatrixDisplay(confusion_matrix=cm)
plt.figure(figsize=(12, 10))  # Adjust width and height for larger plot
cm_display.plot(cmap=plt.cm.Blues)
plt.show()

In [ ]:
cm

In [ ]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')

In [ ]:
from sklearn.metrics import classification_report

# report = classification_report(true_labels, predicted_labels, target_names=class_labels)
# print(report)

report = classification_report(y_test, y_pred, target_names=['sediment', 'sand', 'rubble', 'seagrass', 'seaweed', 'coral', 'rock', 'deeps', 'mangrove', 'land'])
print(report)

In [ ]:
print("Unique labels in y_test:", np.unique(y_test))
print("Unique labels in y_pred:", np.unique(y_pred))

In [ ]:
print("Length of y_test:", len(y_test))
print("Length of y_pred:", len(y_pred))